<a href="https://colab.research.google.com/github/hockyy/food-101-mlcv/blob/main/FoodHockyExplore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive

def import_from_google_drive():
  drive.mount('/content/drive')

import_from_google_drive()

!rm -rf "/content/food-101"
!cp -r "/content/drive/mlcv/" "/content/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import glob
import cv2
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil

def makedir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

def organize_data():
  food_prefix = "/content/food-101"
  food_organized_prefix = "/content/food-101-ok"
  food_path = f"{food_prefix}/*"
  food_directories = glob.glob(food_path)
  food_directories = [dir for dir in food_directories if ("test" not in dir) and ("validate" not in dir) and ("train" not in dir)]
  classes = []
  classes_map = dict()
  for food_directory in food_directories:
    food_name = food_directory.split('/')[3]
    # Map each food name to each index
    current_label = len(classes)
    classes_map[food_name] = current_label
    classes.append(food_name)
    print(f"Organizing {food_directory}")
    filenames = glob.glob(f"{food_directory}/*.jpg")
    train, validate, test = np.split(filenames, [int(len(filenames)*0.8), int(len(filenames)*0.9)])
    print(food_name)
    os.makedirs(f"{food_organized_prefix}/train/{food_name}",exist_ok=True)
    os.makedirs(f"{food_organized_prefix}/validate/{food_name}",exist_ok=True)
    os.makedirs(f"{food_organized_prefix}/test/{food_name}",exist_ok=True)
    for food_file in train:
      food_image_name = food_file.split("/",3)
      try:
        shutil.move(food_file, f"{food_organized_prefix}/train/{food_image_name[-1]}")
      except Exception as e:
        print(e)
    for food_file in validate:
      food_image_name = food_file.split("/",3)
      try:
        shutil.move(food_file, f"{food_organized_prefix}/validate/{food_image_name[-1]}")
      except Exception as e:
        print(e)
    for food_file in test:
      food_image_name = food_file.split("/",3)
      try:
        shutil.move(food_file, f"{food_organized_prefix}/test/{food_image_name[-1]}")
      except Exception as e:
        print(e)

  return classes, classes_map

classes_food_101, classes_map_food_101 = organize_data()

Organizing /content/food-101/spaghetti_carbonara
spaghetti_carbonara
Organizing /content/food-101/takoyaki
takoyaki
Organizing /content/food-101/seaweed_salad
seaweed_salad
Organizing /content/food-101/sashimi
sashimi
Organizing /content/food-101/gnocchi
gnocchi
Organizing /content/food-101/bread_pudding
bread_pudding
Organizing /content/food-101/donuts
donuts
Organizing /content/food-101/garlic_bread
garlic_bread
Organizing /content/food-101/samosa
samosa
Organizing /content/food-101/chicken_curry
chicken_curry


In [26]:
!rm -rf "/content/food-101"

In [30]:
def create_image_generator():
  food_organized_prefix = "/content/food-101-ok"
  train_path = f"{food_organized_prefix}/train"
  validate_path = f"{food_organized_prefix}/validate"
  test_path = f"{food_organized_prefix}/test"
  train_datagen = ImageDataGenerator(rescale=1./255)
  test_datagen = ImageDataGenerator(rescale=1./255)
  validate_datagen = ImageDataGenerator(rescale=1./255)
  image_shape_target = (224,224)
  train_generator = train_datagen.flow_from_directory(
          train_path,
          target_size=image_shape_target,
          color_mode="rgb",
          class_mode="categorical",
          batch_size=32)
  test_generator = test_datagen.flow_from_directory(
          test_path,
          target_size=image_shape_target,
          color_mode="rgb",
          class_mode="categorical",
          batch_size=1)
  validate_generator = validate_datagen.flow_from_directory(
          validate_path,
          target_size=image_shape_target,
          color_mode="rgb",
          class_mode="categorical",
          batch_size=1)
  return train_generator, validate_generator, test_generator

train_gen, test_gen, validate_gen = create_image_generator()

Found 8000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [36]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def create_leNet_model():
  model = keras.Sequential()
  # CNN Layer
  model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(224,224,3)))
  model.add(layers.AveragePooling2D())
  model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
  model.add(layers.AveragePooling2D())
  model.add(layers.Flatten())

  # Dense Layer
  model.add(layers.Dense(units=120, activation='relu'))
  model.add(layers.Dense(units=84, activation='relu'))
  model.add(layers.Dense(units=10, activation = 'softmax'))
  model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return model

import os

def train_model(model, generator, model_name):
  model.save(model_name)
  checkpoint_path = f"{model_name}/cp.ckpt"
  # Create a callback that saves the model's weights
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
  model.fit(train_gen, epochs = 10,callbacks=[cp_callback])



In [43]:
def save_model_to_drive(model_name):
  shutil.copytree(f"/content/{model_name}", f"/content/drive/MyDrive/mlcv/{model_name}")

In [ ]:
from sklearn.metrics._plot.confusion_matrix import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

def predict_and_assess_model(model, test_generator):
  result = model.predict(test_generator)
  print(result)

In [37]:
leNet_model = create_leNet_model()
train_model(leNet_model, train_gen, "lenet")
save_model_to_drive("lenet")


INFO:tensorflow:Assets written to: lenet/assets
Epoch 1/10
250/250 [==============================] - ETA: 0s - loss: 2.1446 - accuracy: 0.2467
Epoch 1: saving model to lenet/cp.ckpt
250/250 [==============================] - 45s 173ms/step - loss: 2.1446 - accuracy: 0.2467
Epoch 2/10
250/250 [==============================] - ETA: 0s - loss: 1.7133 - accuracy: 0.4020
Epoch 2: saving model to lenet/cp.ckpt
250/250 [==============================] - 43s 171ms/step - loss: 1.7133 - accuracy: 0.4020
Epoch 3/10
250/250 [==============================] - ETA: 0s - loss: 1.4388 - accuracy: 0.5077
Epoch 3: saving model to lenet/cp.ckpt
250/250 [==============================] - 43s 172ms/step - loss: 1.4388 - accuracy: 0.5077
Epoch 4/10
250/250 [==============================] - ETA: 0s - loss: 1.0345 - accuracy: 0.6486
Epoch 4: saving model to lenet/cp.ckpt
250/250 [==============================] - 43s 172ms/step - loss: 1.0345 - accuracy: 0.6486
Epoch 5/10
250/250 [========================